# Covid-19 Dashboard

## Timeseries

Here those information of 4 areas of UK will be presented:

Cases relates to new cases by publish date

Hospital relates to new hospital admissions

Deaths relates to new deaths within 28 days of positive test by death date

In [8]:
from uk_covid19 import Cov19API
import json
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as wdg
import time

%matplotlib inline
plt.rcParams['figure.dpi'] = 100

def parse_date(datestring):
    """ Convert a date string into a pandas datetime object """
    return pd.to_datetime(datestring, format="%Y-%m-%d")

def timeseries_callback(button):
    try:
        timeseriesdata_uk=timeseries_uk()
        timeseriesdata_england=timeseries_england()
        timeseriesdata_scotland=timeseries_scotland()
        timeseriesdata_wales=timeseries_wales()
        refresh()
        timeseriesbutton.tooltip=time.strftime("Last downloaded at %d-%m-%Y %H:%M:%S\n", time.localtime())
    except:
        timeseriesbutton.button_style='warning'
        timeseriesbutton.description='Failed'
        time.sleep(5)
        timeseriesbutton.button_style=''
        timeseriesbutton.description='Donwload timeseries data'

def wrangle_data_timeseries(timeseries):    
    datalist=timeseries['data']
    datalist
    
    dates=[dictionary['date'] for dictionary in datalist ]
    dates.sort()    
        
    startdate=parse_date(dates[0])
    enddate=parse_date(dates[-1])
    #print (startdate, ' to ', enddate)
    
    index=pd.date_range(startdate, enddate, freq='D')
    timeseriesdf=pd.DataFrame(index=index, columns=['cases', 'hospital', 'deaths'])
    
    for entry in datalist: # each entry is a dictionary with date, cases, hospital and deaths
        date=parse_date(entry['date'])
        for column in ['cases', 'hospital', 'deaths']:
            # check that nothing is there yet - just in case some dates are duplicated,
            # maybe with data for different columns in each entry
            if pd.isna(timeseriesdf.loc[date, column]): 
                # replace None with 0 in our data 
                value= float(entry[column]) if entry[column]!=None else 0.0
                # this is the way you access a specific location in the dataframe - use .loc
                # and put index,column in a single set of [ ]
                timeseriesdf.loc[date, column]=value
                
    # fill in any remaining "holes" due to missing dates
    timeseriesdf.fillna(0.0, inplace=True)
    
    return timeseriesdf

def timeseries_uk():
    filters = ['areaType=overview']    
        
    structure = {
    "date": "date",
    "cases": "newCasesByPublishDate",
    "hospital": "newAdmissions",
    "deaths": "newDeaths28DaysByDeathDate"    
    }
    
    api = Cov19API(filters=filters, structure=structure)
    timeseries=api.get_json()
    
    with open("timeseries_uk.json", "wt") as OUTF:
        json.dump(timeseries, OUTF)

    timeseriesdf=wrangle_data_timeseries(timeseries)
    
    timeseriesdf.to_pickle("timeseriesdf_uk.pkl") 
    
def timeseries_england():
    filters = ['areaType=nation;areaName=england']    
        
    structure = {
    "date": "date",
    "cases": "newCasesByPublishDate",
    "hospital": "newAdmissions",
    "deaths": "newDeaths28DaysByDeathDate"    
    }
    
    api = Cov19API(filters=filters, structure=structure)
    timeseries=api.get_json()
    
    with open("timeseries_england.json", "wt") as OUTF:
        json.dump(timeseries, OUTF)

    timeseriesdf=wrangle_data_timeseries(timeseries)
    
    timeseriesdf.to_pickle("timeseriesdf_england.pkl") 
    
def timeseries_scotland():
    filters = ['areaType=nation;areaName=scotland']    
        
    structure = {
    "date": "date",
    "cases": "newCasesByPublishDate",
    "hospital": "newAdmissions",
    "deaths": "newDeaths28DaysByDeathDate"    
    }
    
    api = Cov19API(filters=filters, structure=structure)
    timeseries=api.get_json()
    
    with open("timeseries_scotland.json", "wt") as OUTF:
        json.dump(timeseries, OUTF)

    timeseriesdf=wrangle_data_timeseries(timeseries)
    
    timeseriesdf.to_pickle("timeseriesdf_scotland.pkl") 
    
def timeseries_wales():
    filters = ['areaType=nation;areaName=wales']    
        
    structure = {
    "date": "date",
    "cases": "newCasesByPublishDate",
    "hospital": "newAdmissions",
    "deaths": "newDeaths28DaysByDeathDate"    
    }
    
    api = Cov19API(filters=filters, structure=structure)
    timeseries=api.get_json()
    
    with open("timeseries_wales.json", "wt") as OUTF:
        json.dump(timeseries, OUTF)

    timeseriesdf=wrangle_data_timeseries(timeseries)
    
    timeseriesdf.to_pickle("timeseriesdf_wales.pkl")
    
try:
    timeseriesdata_uk=timeseries_uk()
    timeseriesdata_england=timeseries_england()
    timeseriesdata_scotland=timeseries_scotland()
    timeseriesdata_wales=timeseries_wales()

except:
    timeseriesbutton.button_style='warning'
    timeseriesbutton.description='Failed'
    time.sleep(5)
    timeseriesbutton.button_style=''
    timeseriesbutton.description='Donwload timeseries data'

timeseriesdf_uk=pd.read_pickle("timeseriesdf_uk.pkl")
timeseriesdf_england=pd.read_pickle("timeseriesdf_england.pkl")
timeseriesdf_scotland=pd.read_pickle("timeseriesdf_scotland.pkl")
timeseriesdf_wales=pd.read_pickle("timeseriesdf_wales.pkl")
    
series=wdg.SelectMultiple(
    options=['cases', 'hospital', 'deaths'],
    value=['cases', 'hospital', 'deaths'],
    rows=3,
    description='Stats:',
    disabled=False
)

scale=wdg.RadioButtons(
    options=['linear', 'log'],
    description='Scale:',
    disabled=False
)
    
area=wdg.Dropdown(
    options=['UK', 'England', 'Scotland', 'Wales'],
    value='UK',
    description='Area:',
    disabled=False,
)
    
timeseriesbutton=wdg.Button(
    description='Donwload timeseries data',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to download current Public Health England data',
    icon='download' # (FontAwesome names without the `fa-` prefix)
)
    
def refresh():
    current=series.value
    series.value=() # forces the redraw
    series.value=current # now we can change it back
    timeseriesbutton.button_style='success'
    timeseriesbutton.description='Successful'
    time.sleep(3)
    timeseriesbutton.button_style=''
    timeseriesbutton.description='Donwload timeseries data'
       
timeseriesbutton.on_click(timeseries_callback)
    
controls=wdg.VBox([series, area, scale, timeseriesbutton])

def timeseries_graph(gcols, gscale, garea):
    if gscale=='linear':
        logscale=False
    else:
        logscale=True
    ncols=len(gcols)
    if ncols>0 and garea== 'UK':
        timeseriesdf_uk[list(gcols)].plot(logy=logscale)
    elif ncols>0 and garea== 'England':
        timeseriesdf_england[list(gcols)].plot(logy=logscale)
    elif ncols>0 and garea== 'Scotland':
        timeseriesdf_scotland[list(gcols)].plot(logy=logscale)
    elif ncols>0 and garea== 'Wales':
        timeseriesdf_wales[list(gcols)].plot(logy=logscale)
    else:
        print("Click to select data for graph")
        print("(CTRL-Click to select more than one category)")

graph=wdg.interactive_output(timeseries_graph, {'gcols': series, 'gscale': scale, 'garea': area})
    
form=wdg.HBox([graph, controls])
    
display(form)

##  Agedistribution

Here the agedistribution information of England will be presented:

In [7]:
def agedistribution_callback(button):
    try:
        agedistributiondata_england=agedistribution_england()
        refresh1()
        agedistributionbutton.tooltip=time.strftime("Last downloaded at %d-%m-%Y %H:%M:%S\n", time.localtime())
    except:
        agedistributionbutton.button_style='warning'
        agedistributionbutton.description='Failed'
        time.sleep(5)
        agedistributionbutton.button_style=''
        agedistributionbutton.description='Donwload agedistribution data'

def wrangle_data_agedistribution(agedistribution):
    datadic=agedistribution['data'][0] # data['data'] is a list
    
    males=datadic['males']
    females=datadic['females']
    
    ageranges=[x['age'] for x in males] # each entry of males is a dictionary
    
    def min_age(agerange):
        agerange=agerange.replace('+','') # remove the + from 90+
        start=agerange.split('_')[0]
        return int(start)
    
    ageranges.sort(key=min_age)
    
    age_df=pd.DataFrame(index=ageranges, columns=['males','females','total'])
    
    for entry in males: # each entry is a dictionary
        ageband=entry['age'] # our index position
        age_df.loc[ageband, 'males']=entry['value']
    
    for entry in females:
        ageband=entry['age']
        age_df.loc[ageband, 'females']=entry['value']
               
    # this is straightforward
    age_df['total']=age_df['males']+age_df['females']   
    
    return age_df

def agedistribution_england():
    filters = ['areaType=nation;areaName=england']

    # values here are the names of the PHE metrics
    structure = {
        "males": "maleCases",
        "females": "femaleCases"
    }

    api = Cov19API(filters=filters, structure=structure)
    agedistribution=api.get_json()
    
    with open("agedistribution_england.json", "wt") as OUTF:
        json.dump(agedistribution, OUTF) 
        
    age_df=wrangle_data_agedistribution(agedistribution)
        
    age_df.to_pickle("age_df_england.pkl")
    
try:    
    agedistributiondata_england=agedistribution_england()
except:
    agedistributionbutton.button_style='warning'
    agedistributionbutton.description='Failed'
    time.sleep(5)
    agedistributionbutton.button_style=''
    agedistributionbutton.description='Donwload agedistribution data'

age_df_england=pd.read_pickle("age_df_england.pkl")
    
agecols=wdg.SelectMultiple(
    options=['males', 'females', 'total'], # options available
    value=['males', 'females'], # initial value
    rows=3, # rows of the selection box
    description='Sex',
    disabled=False
)
    
agedistributionbutton=wdg.Button(
    description='Donwload agedistribution data',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to download current Public Health England data',
    icon='download' # (FontAwesome names without the `fa-` prefix)
 )
    
def refresh1():
    current=agecols.value
    agecols.value=() # forces the redraw
    agecols.value=current # now we can change it back
    agedistributionbutton.button_style='success'
    agedistributionbutton.description='Successful'
    time.sleep(3)
    agedistributionbutton.button_style=''
    agedistributionbutton.description='Donwload agedistribution data'
        
agedistributionbutton.on_click(agedistribution_callback)
        
controls1=wdg.VBox([agecols, agedistributionbutton])
    
def age_graph(graphcolumns):
    # our callback function.
    ncols=len(graphcolumns)
    if ncols>0:
        age_df_england.plot(kind='bar', y=list(graphcolumns)) # graphcolumns is a tuple - we need a list
    #elif ncols>0 and garea=='Scotland':
    #age_df_scotland.plot(kind='bar', y=list(graphcolumns)) # graphcolumns is a tuple - we need a list
    else:
        # if the user has not selected any column, print a message instead
        print("Click to select data for graph")
        print("(CTRL-Click to select more than one category)")  
    
output=wdg.interactive_output(age_graph, {'graphcolumns': agecols})
    
form1=wdg.HBox([output, controls1])
    
display(form1)

# register the callback function with the button
# this is an iPython function that generalises print for Jupyter Notebooks; we use it to 
# display the widgets

Peiheng Yang 200872232